In [ ]:
from q2_time.compare_models import compare_config
from q2_time.engineer_features import transform_features
from q2_time.model import fit_n_predict_model
from q2_time.simulate_data import simulate_data

%load_ext autoreload
%autoreload 2

## Simulate datasets

In [ ]:
ft, md = simulate_data(100)

In [ ]:
ft.head()

In [ ]:
md.head()

## Transform features

In [ ]:
ft.describe()

In [ ]:
ft_t = transform_features(ft, "clr")
ft_t.describe()

## Train & compare models

In [ ]:
## Static model
lin_reg, lin_config, _, _ = fit_n_predict_model(
    md, ft, "age_days", "host_id", "LinRegressor"
)

rf_reg, rf_config, _, _ = fit_n_predict_model(
    md, ft, "age_days", "host_id", "RFRegressor"
)

In [ ]:
compare_config({"linreg": lin_config, "rf": rf_config})